In [ ]:
import numpy as np
import os
import pickle
from SleepingThread import Timer,Saver
from SleepingThread.qsar import descriptors as descr
sel_folder = "/home/unknown/SCW/datasets/bzr/selection"

sgt = descr.MolSegmentator()
# ElVDW
sgt.setMol(sel_folder+"/1",prop_type="El")
sgt.drawSurface(draw_prop=True)
sgt.generateMolImage(20)
sgt.drawMolImage()

In [ ]:
import numpy as np
import os
import pickle
from SleepingThread import Timer,Saver
from SleepingThread.qsar import descriptors as descr
sel_folder = "/home/unknown/SCW/datasets/bzr/selection"

class ArgsGenerator(object):
    def __init__(self,sel_folder):
        self.sel_folder = sel_folder
        self.file_num = 1
        
    def next(self):
        if os.path.isfile(sel_folder+"/"+str(self.file_num)+".mol"):
            self.file_num += 1
            return sel_folder+"/"+str(self.file_num-1)
        else:
            return None

generator = ArgsGenerator(sel_folder)
        
saver = Saver(folder=sel_folder+"/algdata/mol_segm",generator=generator,sec=600,verbose=1,zip_output=True)

sgt_list = []

gl_timer = Timer()

while saver.next(sgt_list):
    args = saver.getArgs()
    print args
    
    loc_timer = Timer()
    sgt = descr.MolSegmentator()
    sgt.setMol(args)
    sgt.generateSegmentation(scales=[10.0],verbose=1)
    print sgt.n_segments, loc_timer, gl_timer
    sgt_list.append(sgt)

In [ ]:
import sys
for ind,sgt in enumerate(sgt_list):
    sys.stdout.write("\r"+str(ind))
    sgt.generateSegmentImages(10)
    sgt.setSegmentPropertyType("average")

data = descr.getDataFromMolSegmentatorList(sgt_list,verbose=1)

In [ ]:
from SleepingThread.qsar.qsarmodel import QSARModel1
from SleepingThread.ml import *
import numpy as np
from sklearn.model_selection import cross_val_score, ShuffleSplit, StratifiedKFold, RepeatedStratifiedKFold, \
    cross_validate
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import sys

def fun(kernel="rbf",n_image_clusters=1):
    output_y = []
    output_x = []
    
    output2_y = []
    output2_x = []
    
    timer = Timer()
    
    for n_prop_clusters in xrange(1,30,2):
        sys.stdout.write("\rn_prop_clusters: "+str(n_prop_clusters))
        
        target = np.loadtxt(sel_folder+"/values")
        estimator = SVC(C=1.0,class_weight={ 1:float(np.sum(target==-1))/float(np.sum(target==1)) },kernel='linear')
        #estimator = SVC(C=1.0,kernel='linear')
        qm = QSARModel1(estimator=estimator,\
                n_image_clusters=n_image_clusters,\
                n_prop_clusters=n_prop_clusters)
        mat = qm.getMatrix(data)
        
        sc = StandardScaler()
        mat = sc.fit_transform(mat)
        
        trainer = SVC(C=1.0,class_weight={ 1:float(np.sum(target==-1))/float(np.sum(target==1)) },kernel=kernel)
        trainer.fit(mat,target)
        
        #print "All Weighted: ",weighted_accuracy(trainer,mat,target)
        #print "All Score: ",trainer.score(mat,target)
        
        #from SleepingThread.qsar.graphics import drawBar
        #drawBar(target-estimator.predict(mat),n_segments=15)
    
        trainer = SVC(C=1.0,class_weight={ 1:float(np.sum(target==-1))/float(np.sum(target==1)) },kernel=kernel)
        cv = RepeatedStratifiedKFold(n_splits=5,random_state=0,n_repeats=30)
        #cv = ShuffleSplit(n_splits=100,test_size=0.2,random_state=0)
        res = cross_validate(trainer,mat,target,cv=cv,scoring={'accuracy':'accuracy','wa':weighted_accuracy},return_train_score=True)
    
        output_y.append(np.average(res["test_wa"]))
        output_x.append(n_prop_clusters)
        
        output2_y.append(np.average(res["train_wa"]))
        output2_x.append(n_prop_clusters)
        
        #print "Weighted test: ", np.average(res["test_wa"])
        #print "Weighted train: ", np.average(res["train_wa"])
        #print "Accuracy test: ", np.average(res["test_accuracy"])
        #print "Accuracy train: ", np.average(res["train_accuracy"])
        
    sys.stdout.write("\nTime: "+str(timer)+"\n")
    
    return output_x,output_y,output2_x,output2_y

from matplotlib import pyplot as plt

def draw_fun(kernel="rbf",n_image_clusters=1):
    print "kernel: ",kernel," | n_image_clusters: ",n_image_clusters
    output_x,output_y,output2_x,output2_y = fun(kernel,n_image_clusters)
    plt.plot(output_x,output_y,color="blue")
    plt.plot(output2_x,output2_y,color="red")
    plt.show()

In [ ]:
for n_image_clusters in xrange(1,5):
    draw_fun("linear",n_image_clusters)
    draw_fun("rbf",n_image_clusters)

In [ ]:
"""

"""

In [ ]:
from SleepingThread.qsar import descriptors as descr
sel_folder = "/home/unknown/SCW/datasets/bzr/selection"
sgt = descr.MolSegmentator()
sgt.setMol(sel_folder+"/1")
sgt.generateSegmentation(scales=[10.0],verbose=1)
sgt.drawSegmentationScores()
print sgt.n_segments
sgt.drawSurface(draw_segm=True,draw_segm_centers=True)

In [ ]:
sgt.drawSegments(start=0,end=5,draw_prop=True,draw_segm_centers=True)

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure()
plt.table(cellText=[["asdasd","sdfsdfsd"],["zxczxkfms",",mvnblkdfg"]],colLabels=["A","B"],rowLabels=["C","D"])
fig.axes[0].axis('off')
plt.show()

In [ ]:
sgt = descr.MolSegmentator()
sgt.setSurface(points_list[0],mesh_index_list[0],[prop_list[0]])
sgt.generateSegmentation(18,scales=[10.0])
sgt.generateSegmentImages(10)
sgt.generateMolImage(10)

In [ ]:
from SleepingThread import ml
from sklearn.cluster import KMeans
import numpy as np
np.random.seed(0)
kmeans = KMeans(n_clusters=2)
data = np.random.random(30).reshape(-1,1)
scree = ml.ScreeSearch(kmeans,{"n_clusters":[2,3,4,5,6,7]})
scree.fit(data)